In [1]:
import numpy as np

In [2]:
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import matplotlib.patches as patches
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))
    
    

In [4]:
#diff = np.float32(0.033068776)
diff = np.float32(0.03306878)
global_min = -31*diff
global_max = 31*diff

dom_maxs = np.array([global_max]*3, dtype=np.float64)
dom_mins = np.array([global_min]*3, dtype=np.float64)
tau = 0.2/12600 # 0.2 times mean interparticle separation
#buffer_tau = diff*5./150.
Ngrid = 62

In [5]:
import sparkhpc

In [6]:
ncores = 8

In [7]:
import os
os.environ['SPARK_HOME'] = '/home/ics/roskar/spark'
os.environ['SPARK_DRIVER_MEMORY'] = '16G'

In [8]:
import findspark
findspark.init()

In [9]:
import sparkhpc

In [10]:
clusterid=0

if clusterid is None: 
    sj = sparkhpc.sparkjob.sparkjob(ncores=8,
                                memory=50000,
                                walltime='08:00', 
                                cores_per_executor=4, 
                                memory_per_executor=50000)
    sj.wait_to_start()
else:
    sj = sparkhpc.sparkjob.sparkjob(clusterid=clusterid)

In [11]:
sj.show_clusters()

ClusterID,Job ID,Number of cores,Status,Spark UI,Spark URL
0,1145,8,running,http://172.19.1.158:8080,spark://x09y02:7077


In [12]:
sc = sparkhpc.start_spark(master=sj.master_url, spark_conf='../conf', 
                          profiling=False, executor_memory='30000M', graphframes_package='graphframes:graphframes:0.3.0-spark2.0-s_2.11')

In [13]:
sc.setCheckpointDir('file:///zbox/data/roskar/work/checkpoint')

In [14]:
import spark_fof
reload(spark_fof.spark_fof)
reload(spark_fof)
path = '/zbox/trove/euclid/2Tlc-final/'

nMinMembers = 8
nBins = 62
minblock = 30
maxblock = 32
fof_analyzer = spark_fof.spark_fof.LCFOFAnalyzer(sc, path, nMinMembers, nBins, tau, dom_mins, dom_maxs, blockids=range(minblock,maxblock), buffer_tau=tau*2)

spark_fof: Number of input files:  8
spark_fof: Total number of particles:  594482071


In [15]:
fof_analyzer.particle_rdd.setName('particle_rdd')

particle_rdd PythonRDD[5] at RDD at PythonRDD.scala:48

In [16]:
fof_analyzer.merged_rdd.cache()

spark_fof: domain group mapping build took 71.411676 seconds


UnboundLocalError: local variable 'create_map_dict' referenced before assignment

In [ ]:
fof_analyzer.merged_rdd.first()

In [17]:
# get the mapping

In [18]:
mapping = fof_analyzer._get_level_map()

spark_fof: domain group mapping build took 71.481189 seconds


In [19]:
mapping.cache()

PythonRDD[225] at RDD at PythonRDD.scala:48

In [21]:
def decode(gid): 
    pid_p = gid >> 32
    gid_p = gid ^ (pid_p << 32)
    return pid_p, gid_p

In [20]:
def create_map_dict(iterator):
    group_map_dict = {g:g_p for g,g_p in iterator}
    yield group_map_dict

In [36]:
partitioned_mapping = (mapping.map(lambda (g,g_p): (decode(g)[0], (g,g_p)))
                              .partitionBy(8)
                              .map(lambda (k,v): v, preservesPartitioning=True)
                              .mapPartitions(create_map_dict, preservesPartitioning=True)).cache()

In [37]:
partitioned_mapping.setName('mapping').count()

8

In [29]:
combined = partitioned_mapping + fof_analyzer.fof_rdd 

In [33]:
def remap_local_groups(iterator):    
    group_mapping_dict = iterator.next()    
    for p_arr in iterator:
        spark_fof.spark_fof_c.remap_gid_partition_cython(p_arr, group_mapping_dict)
        yield p_arr

In [35]:
combined.mapPartitions(remap_local_groups).count()

61

In [16]:
%time fof_analyzer.finalize_groups()

spark_fof <timing>: domain group mapping build took 77.049951 seconds
total groups:  3099992
spark_fof: Final group map build took 1.884542 seconds
CPU times: user 24.1 s, sys: 992 ms, total: 25.1 s
Wall time: 14min 16s


PythonRDD[236] at RDD at PythonRDD.scala:48

In [17]:
sj.stop()

INFO:sparkhpc.sparkjob:
